# PUNTO 7. Micro-Laboratorio (Ejercicio Práctico)

**Consigna:**

Usando las `reviews` y las funciones de preprocesamiento de clases previas (o volviendo a procesarlas ahora):
1.  Asegúrate de tener la lista de `reviews_preprocesadas` (cada elemento es un string con los lemas unidos por espacios). Si no la tenés, generála usando la función `preprocesar_texto_para_vectorizar` sobre las `reviews` originales.
2.  **Crear Matriz BoW:**
    *   Instancia un `CountVectorizer`.
    *   Aplícalo a las `reviews_preprocesadas` usando `fit_transform()`.
    *   Obtén el vocabulario (`get_feature_names_out()`).
    *   Crea un DataFrame de Pandas para visualizar la matriz BoW.
3.  **Crear Matriz TF-IDF:**
    *   Instancia un `TfidfVectorizer`.
    *   Aplícalo a las **mismas** `reviews_preprocesadas` usando `fit_transform()`.
    *   **Importante:** Para comparar fácil, puedes pasarle el vocabulario aprendido por el CountVectorizer al TfidfVectorizer usando el argumento `vocabulary=`. O viceversa. La idea es que ambas matrices usen las mismas columnas en el mismo orden.
    *   Crea un DataFrame de Pandas para visualizar la matriz TF-IDF (redondea los valores a 3 decimales).
4.  **Analizar:**
    *   Imprime ambas matrices.
    *   Elige una o dos reviews. ¿Qué palabras tienen los pesos más altos en TF-IDF para esa review? ¿Coincide con lo que esperarías que sean las palabras clave de esa review?
    *   Busca alguna palabra que tenga un conteo > 0 en BoW pero un peso TF-IDF relativamente bajo. ¿Por qué podría ser? (Pista: ¿aparece en muchas reviews?).

# **Preprocesamiento**

In [6]:
import re  # Módulo para trabajar con expresiones regulares (limpieza de texto).
import nltk  # Toolkit de procesamiento de lenguaje natural.
from nltk.corpus import stopwords  # Carga las "stopwords" (palabras vacías) de NLTK.
from nltk.stem import SnowballStemmer  # Uso el stemmer Snowball para español (reducción de palabras a su raíz).
from sklearn.feature_extraction.text import CountVectorizer  # Importamos el vectorizador BoW (bolsa de palabras) de scikit-learn.
import pandas as pd  # Importamos pandas para manejar la matriz como DataFrame y visualizarla más lindo
from sklearn.feature_extraction.text import TfidfVectorizer  # Importamos el vectorizador TF-IDF desde scikit-learn.

In [7]:
nltk.download('stopwords')  # Descargar el listado de stopwords en español
stop_words = set(stopwords.words('spanish'))  # Creamos un set de palabras vacías en español.
stemmer = SnowballStemmer('spanish')  # Instanciamos el stemmer para español (convierte palabras a su raíz).

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# --- Función de preprocesamiento: recibe un texto y devuelve una versión limpia, lematizada y lista para vectorizar ---

def preprocesar_texto_para_vectorizar(texto):
    texto = texto.lower()  # Paso 1: pasamos todo a minúsculas para uniformizar.
    texto = re.sub(r"[^\w\s]", "", texto)  # Paso 2: eliminamos puntuación y caracteres raros.
    tokens = texto.split()  # Paso 3: tokenizamos (separamos por espacios).
    tokens = [stemmer.stem(palabra) for palabra in tokens if palabra not in stop_words]

    # Paso 4: aplicamos stemming a cada palabra, ignorando las stopwords.
    return " ".join(tokens)  # Paso 5: volvemos a unir las palabras preprocesadas en un único string.

In [9]:
# --- Dataset de reviews original (cinco frases sobre películas/documentales) ---

reviews = [
    "Una película emocionante con actuaciones brillantes. ¡Me encantó!",
    "Muy aburrida y lenta. El guión era predecible y los actores no convencían.",
    "Los efectos especiales fueron impresionantes, pero la historia dejaba mucho que desear.",
    "¡Qué gran comedia! Me reí sin parar durante toda la película.",
    "Un documental necesario que aborda temas importantes con profundidad y sensibilidad."
]

In [10]:
# Aplicamos la función de preprocesamiento a cada review del dataset original
reviews_preprocesadas = [preprocesar_texto_para_vectorizar(r) for r in reviews]

# **Matriz BoW**

In [11]:
# --- Vectorización BoW (Bag of Words): transforma texto en una matriz de conteos de palabras ---

vectorizer_bow = CountVectorizer()  # Instanciamos el vectorizador (sin parámetros extra: toma el vocabulario que aparece)
matriz_bow = vectorizer_bow.fit_transform(reviews_preprocesadas)
# Ajustamos (fit) y transformamos las reviews preprocesadas al formato BoW. Resultado: una matriz sparse (dispersa).

In [12]:
vocabulario = vectorizer_bow.get_feature_names_out()
# Extraemos las palabras (features) del vocabulario aprendido, en el orden en que están en las columnas de la matriz.

In [13]:
df_bow = pd.DataFrame(matriz_bow.toarray(), columns=vocabulario)
# Convertimos la matriz dispersa a un array denso (toarray), y lo envolvemos en un DataFrame para que sea legible.

In [14]:
print("Matriz BoW:")  # Título
print(df_bow)  # Mostramos la matriz BoW completa con conteos por review y por palabra

Matriz BoW:
   abord  aburr  actor  actuacion  brillant  comedi  convenc  dej  des  \
0      0      0      0          1         1       0        0    0    0   
1      0      1      1          0         0       0        1    0    0   
2      0      0      0          0         0       0        0    1    1   
3      0      0      0          0         0       1        0    0    0   
4      1      0      0          0         0       0        0    0    0   

   documental  ...  lent  necesari  par  pelicul  predec  profund  rei  \
0           0  ...     0         0    0        1       0        0    0   
1           0  ...     1         0    0        0       1        0    0   
2           0  ...     0         0    0        0       0        0    0   
3           0  ...     0         0    1        1       0        0    1   
4           1  ...     0         1    0        0       0        1    0   

   sensibil  tem  tod  
0         0    0    0  
1         0    0    0  
2         0    0    0  
3 



---


*Cada número indica cuántas veces aparece una palabra (columna) en una review (fila).*

-- Breve interpretación:

**Fila** = una review (hay 5).

**Columna**= una palabra preprocesada (hay 29).

Ej: En la review 0, la palabra "actuacion" aparece 1 vez, "aburr" aparece 0 veces.

*Muchas celdas son ceros: es normal en matrices de texto porque no todas las palabras están en todas las reviews*



---



# **Matriz TF-IDF**

In [15]:
vectorizer_tfidf = TfidfVectorizer(vocabulary=vocabulario)
# Instanciamos el vectorizador TF-IDF, **forzando** a que use el mismo vocabulario que el CountVectorizer

In [16]:
matriz_tfidf = vectorizer_tfidf.fit_transform(reviews_preprocesadas)
# Ajustamos (fit) y transformamos las reviews preprocesadas a su representación TF-IDF.
# Devuelve una matriz dispersa con los pesos TF-IDF (cuánto "pesa" cada término según frecuencia y rareza).

df_tfidf = pd.DataFrame(matriz_tfidf.toarray(), columns=vocabulario)
# Convertimos la matriz dispersa a un array denso (toarray) y la envolvemos en un DataFrame para que sea legible.


In [17]:
df_tfidf = df_tfidf.round(3)
# Redondeamos los valores a 3 decimales para que no se vea una ensalada de floats

In [18]:
print("\n Matriz TF-IDF:")  # Título para distinguir que ahora viene la matriz TF-IDF.
print(df_tfidf)  # Mostramos el DataFrame con los valores TF-IDF por review y por término.


 Matriz TF-IDF:
   abord  aburr  actor  actuacion  brillant  comedi  convenc    dej    des  \
0  0.000  0.000  0.000      0.464     0.464   0.000    0.000  0.000  0.000   
1  0.000  0.408  0.408      0.000     0.000   0.000    0.408  0.000  0.000   
2  0.000  0.000  0.000      0.000     0.000   0.000    0.000  0.408  0.408   
3  0.000  0.000  0.000      0.000     0.000   0.421    0.000  0.000  0.000   
4  0.378  0.000  0.000      0.000     0.000   0.000    0.000  0.000  0.000   

   documental  ...   lent  necesari    par  pelicul  predec  profund    rei  \
0       0.000  ...  0.000     0.000  0.000    0.374   0.000    0.000  0.000   
1       0.000  ...  0.408     0.000  0.000    0.000   0.408    0.000  0.000   
2       0.000  ...  0.000     0.000  0.000    0.000   0.000    0.000  0.000   
3       0.000  ...  0.000     0.000  0.421    0.339   0.000    0.000  0.421   
4       0.378  ...  0.000     0.378  0.000    0.000   0.000    0.378  0.000   

   sensibil    tem    tod  
0     0.000



---


*Cada número representa cuánto "pesa" o importa una palabra (columna) en una review (fila), según:*

**TF**: frecuencia en la review.

**IDF**: rareza en el conjunto total (menos común = más peso).

--- Interpretación rápida:
+ Valor alto = palabra importante y poco frecuente en el resto.

+ Valor bajo = palabra que aparece mucho o es irrelevante (como “película”).


--- Ejemplo:
+ En la review 0, "actuacion" y "brillant" tienen peso 0.464 → son claves en esa frase.

 "pelicul" aparece varias veces en muchas reviews, por eso tiene peso menor (ej: 0.374 o menos).


*Esto refina el BoW: no es solo cuántas veces aparece una palabra, sino cuánto valor aporta al texto.*


---



# **Análisis cualitativo**

In [20]:
# Elegimos la review 0 (índice 0)
print("\n Palabras con mayor peso en TF-IDF para review 0:")
review_0 = df_tfidf.iloc[0]  # Extrae los pesos TF-IDF de la primera review
print(review_0.sort_values(ascending=False).head(5))  # Muestra las 5 palabras con mayor peso

# Buscar palabra con alto conteo en BoW pero bajo TF-IDF
print("\n Palabras frecuentes en BoW pero con bajo peso TF-IDF:")
frecuentes = (df_bow.sum(axis=0) > 0)  # Palabras que aparecen al menos una vez (más laxo)
pesos_bajos = (df_tfidf.mean(axis=0) < 0.3)  # Umbral más alto para considerar "bajo peso"
candidatas = df_bow.columns[frecuentes & pesos_bajos]  # Palabras frecuentes pero poco informativas
print(candidatas.tolist())  # Imprime la lista de esas palabras


 Palabras con mayor peso en TF-IDF para review 0:
brillant     0.464
actuacion    0.464
emocion      0.464
encant       0.464
pelicul      0.374
Name: 0, dtype: float64

 Palabras frecuentes en BoW pero con bajo peso TF-IDF:
['abord', 'aburr', 'actor', 'actuacion', 'brillant', 'comedi', 'convenc', 'dej', 'des', 'documental', 'efect', 'emocion', 'encant', 'especial', 'gran', 'guion', 'histori', 'import', 'impresion', 'lent', 'necesari', 'par', 'pelicul', 'predec', 'profund', 'rei', 'sensibil', 'tem', 'tod']


*Las palabras con mayor peso **en TF-IDF** para la review 0 (brillant, actuacion, emocion, encant, pelicul) reflejan con precisión los conceptos clave de esa opinión positiva, ya que son términos distintivos que no se repiten tanto en otras reviews.*


-

*En contraste, se identificaron varias palabras que aparecen frecuentemente en el corpus (alto conteo **en BoW)** pero con bajo peso TF-IDF. Esto se debe a que, al estar presentes en **múltiples** textos, su valor como discriminador baja. Incluso algunas palabras que destacan en una review (como actuacion o emocion) pueden tener un promedio bajo de TF-IDF si están dispersas en el resto del conjunto.*



---


***Conclusión: TF-IDF permite resaltar lo más representativo de cada review individual, mientras que BoW muestra cantidad pero no relevancia.***

